# Crime Trail Blazers Data

In [1]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [2]:
# Name of the CSV file
file15 = 'Resources/2015_Portland_Crime_Data.csv'
file16 = 'Resources/2016_Portland_Crime_Data.csv'
file17 = 'Resources/2017_Portland_Crime_Data.csv'
file18 = 'Resources/2018_Portland_Crime_Data.csv'
file19 = 'Resources/2019YTD_Portland_Crime_Data.csv'
nba_data ='Resources/portland.games.stats.csv'

In [3]:
# The correct encoding must be used to read the CSV in pandas
df_2015 = pd.read_csv(file15, encoding="ISO-8859-1")
df_2016 = pd.read_csv(file16, encoding="ISO-8859-1")
df_2017 = pd.read_csv(file17, encoding="ISO-8859-1")
df_2018 = pd.read_csv(file18, encoding="ISO-8859-1")
df_2019 = pd.read_csv(file19, encoding="ISO-8859-1")
nba_data = pd.read_csv(nba_data, encoding="ISO-8859-1")

## Portland Trailblazers Stats 2015-18

In [4]:
nba_data.head()

,Unnamed: 0,Team,Game,Date,Home,Opponent,WINorLOSS,TeamPoints,OpponentPoints,FieldGoals,...,Opp.FreeThrows,Opp.FreeThrowsAttempted,Opp.FreeThrows.,Opp.OffRebounds,Opp.TotalRebounds,Opp.Assists,Opp.Steals,Opp.Blocks,Opp.Turnovers,Opp.TotalFouls
0,1108,POR,1,10/29/2014,Home,OKC,W,106,89,39,...,21,26,0.808,14,43,19,9,5,13,28
1,2108,POR,2,10/31/2014,Away,SAC,L,94,103,34,...,31,35,0.886,14,49,14,6,2,10,24
2,3108,POR,3,11/2/2014,Home,GSW,L,90,95,37,...,17,21,0.810,7,41,22,11,7,17,16
3,4108,POR,4,11/4/2014,Home,CLE,W,101,82,37,...,11,14,0.786,13,41,18,5,3,14,19
4,5108,POR,5,11/6/2014,Home,DAL,W,108,87,40,...,16,16,1.000,8,34,14,6,4,7,17


In [5]:
# add Day of Week
nba_data['DOW']= pd.to_datetime(nba_data['Date']).dt.dayofweek

In [6]:
# Change date format and reduce dates to 2015 - 2018
nba_data['Date'] = pd.to_datetime(nba_data['Date'])
start_date = '01/01/2015'
end_date = '12/31/2018'
mask = (nba_data['Date'] >= start_date) & (nba_data['Date'] <= end_date)
nba_data = nba_data.loc[mask]

In [7]:
# Created a new dataframe using the data_df to display these columns:
new_nba_df = nba_data[['Date','Team','Game','Home','WINorLOSS', 'DOW']]

## Crime Data

In [8]:
# Concatinate all crime data
concat_crime_df = pd.concat([df_2015, df_2016, df_2017, df_2018, df_2019])

In [9]:
# add Day of Week
concat_crime_df['DOW']= pd.to_datetime(concat_crime_df['OccurDate']).dt.dayofweek

In [10]:
# Created a new dataframe using the data_df to display these columns:
new_crime_df = concat_crime_df[['OccurDate','Neighborhood','OffenseCategory','DOW']]

In [11]:
# Rename date field
new_crime_df = new_crime_df.rename(columns={"OccurDate":"Date"})

In [12]:
# Filter out reports of crimes prior to 2015
new_crime_df['Date'] = pd.to_datetime(new_crime_df['Date'])
start_date = '01/01/2015'
end_date = '12/31/2018'
mask = (new_crime_df['Date'] >= start_date) & (new_crime_df['Date'] <= end_date)
new_crime_df = new_crime_df.loc[mask]
new_crime_df

,Date,Neighborhood,OffenseCategory,DOW
0,2015-05-12,Piedmont,Assault Offenses,1
1,2015-05-01,Buckman West,Assault Offenses,4
2,2015-05-01,University Park,Assault Offenses,4
3,2015-05-01,Madison South,Assault Offenses,4
4,2015-05-01,Madison South,Kidnapping/Abduction,4
...,...,...,...,...
48520,2018-12-05,NaN,Fraud Offenses,2
48531,2018-09-10,NaN,Fraud Offenses,0
48533,2017-03-04,NaN,Fraud Offenses,5
48541,2016-03-01,NaN,Fraud Offenses,1


In [13]:
# Create new dataframe specific to Date occurances
df_dow = new_crime_df[["Date", "DOW"]]

In [14]:
# Run for loop to count number of crimes for each date
unique_dates = list(df_dow["Date"].unique())

listy = []
for i in unique_dates:
    holdy = df_dow.loc[df_dow["Date"] == i].reset_index(drop=True)
    holdy1 = holdy["Date"].count()
    holder_dictionary = {"Date": i,
                         "Num_times": holdy1,
                         "DOW": holdy["DOW"][0]}
    listy.append(holder_dictionary)

In [15]:
# Create datafrom of the daily crime count
crime_count_df = pd.DataFrame(listy)
crime_count_df

,Date,Num_times,DOW
0,2015-05-12,161,1
1,2015-05-01,195,4
2,2015-05-02,176,5
3,2015-05-03,130,6
4,2015-05-04,143,0
...,...,...,...
1454,2018-12-28,152,4
1455,2018-12-29,129,5
1456,2018-12-30,143,6
1457,2018-12-31,136,0


## Trail Blazers and Crime Data

In [16]:
# Merge Trail Blazer and Crime Data
merge_table = pd.merge(new_crime_df, new_nba_df, on="Date", how="left")

In [17]:
# Modify merged table to add columns for further analysis
# Add Year, Month and Month_Year data
merge_table["HasGame"] = merge_table['Game'] > 0
merge_table["GameStatus"] = merge_table["Home"] + merge_table["WINorLOSS"] 
merge_table["GameStatus"] = merge_table["GameStatus"].fillna('NoGame') 
merge_table["Year"] = pd.DatetimeIndex(merge_table["Date"]).year
merge_table["Month"] = pd.DatetimeIndex(merge_table["Date"]).month
merge_table["Month_Year"] = pd.to_datetime(merge_table["Date"]).dt.to_period('M')

In [18]:
# Created a new dataframe to display these columns:
month_crime_df = merge_table[['Date','Year','Month','Month_Year']]
month_crime_df.head()

,Date,Year,Month,Month_Year
0,2015-05-12,2015,5,2015-05
1,2015-05-01,2015,5,2015-05
2,2015-05-01,2015,5,2015-05
3,2015-05-01,2015,5,2015-05
4,2015-05-01,2015,5,2015-05


In [19]:
# Export the crime count into a csv
month_crime_df.to_csv('Output/month_year_crime.csv')

In [20]:
grouped_table = merge_table.groupby(['DOW_x', 'GameStatus']).agg(
    {'OffenseCategory':'count',
     'Date':'nunique'
    }
)
grouped_table['AverageDailyCrimes'] = grouped_table['OffenseCategory']/grouped_table['Date']
grouped_table

OffenseCategory  Date  AverageDailyCrimes
DOW_x GameStatus                                           
0     AwayL                  2225    17          130.882353
      AwayW                  1351     8          168.875000
      HomeL                   569     4          142.250000
      HomeW                   832     8          104.000000
      NoGame                25683   172          149.319767
1     AwayL                  1453     8          181.625000
      AwayW                  1277     8          159.625000
      HomeL                  1423     9          158.111111
      HomeW                  2077    14          148.357143
      NoGame                24231   169          143.378698
2     AwayL                  2492    19          131.157895
      AwayW                   697     6          116.166667
      HomeL                  1018     8          127.250000
      HomeW                  2474    20          123.700000
      NoGame                24282   155          156.658065
3     AwayL                  1010     8          126.250000
      AwayW                   187     1          187.000000
      HomeL                  1023     7          146.142857
      HomeW                  2468    18          137.111111
      NoGame                26166   175          149.520000
4     AwayL                  1873    15          124.866667
      AwayW                  1763    12          146.916667
      HomeL                  1165     7          166.428571
      HomeW                  1542    11          140.181818
      NoGame                26503   164          161.603659
5     AwayL                   975    10           97.500000
      AwayW                  1656    10          165.600000
      HomeL                  1072     8          134.000000
      HomeW                  2663    20          133.150000
      NoGame                24414   160          152.587500
6     AwayL                  1256     9          139.555556
      AwayW                  1756    16          109.750000
      HomeL                   683     6          113.833333
      HomeW                  1094     7          156.285714
      NoGame                23536   170          138.447059

In [21]:
# Export Day of Week Crime table
grouped_table.to_csv('Output/day_of_week_table.csv')

In [22]:
total_table = merge_table.groupby(['GameStatus']).agg(
    {'OffenseCategory':'count',
     'Date':'nunique'
    }
)
total_table['AverageCrime'] = total_table['OffenseCategory']/total_table['Date']

In [23]:
# Export Overal Crime table
total_table.to_csv('Output/game_status_table.csv')

In [24]:
# Merge Trail Blazer and Crime Count
count_table = pd.merge(crime_count_df, new_nba_df, on="Date", how="left")
count_table["HasGame"] = count_table['Game'] > 0
count_table["Month_Year"] = pd.to_datetime(count_table["Date"]).dt.to_period('M')
count_table = count_table[['Date','Num_times','HasGame','Month_Year']]

In [25]:
count_table.to_csv('Output/gameday_table.csv')